In [60]:
import os
import sys
import numpy as np
import torchvision.models as models

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

import torchvision
from torchvision.transforms import Compose, Resize, Normalize, ToTensor, RandomCrop, RandomHorizontalFlip, CenterCrop, Pad
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [4]:
transform_train = Compose([
    RandomCrop(32, padding=4),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = Compose([
    ToTensor(),
    Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
if device == 'cuda':
    cudnn.benchmark = True
    
net = torch.load('../models/target/cifar10.vgg19.pth')
net = net.to(device)

In [6]:
sys.path.insert(0, '/home/brendalf/Documents/projects/gan-attack/src')

In [7]:
# from target.custom import CustomNN
from oraculo.train import train_network
from oraculo.evaluate import evaluate_network

In [8]:
import tqdm

# cifar 10

In [40]:
#train_network(network, trainloader, output_path, 50)

In [9]:
evaluate_network(net, testloader)

  0%|          | 0/313 [00:00<?, ?it/s]

Evaluating model...


100%|██████████| 313/313 [00:02<00:00, 120.15it/s]

Accuracy: 93 %


93.64

# stl 10

In [64]:
#transform_train = Compose([
#    RandomCrop(32, padding=4),
#    RandomHorizontalFlip(),
#    ToTensor(),
#    Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#])

transform_test = Compose([
    Resize((30,30)),
    Pad(1, fill=1),
    ToTensor(),
    Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#trainset = torchvision.datasets.STL10(root='../data', split='train', download=True, transform=transform_train)
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.STL10(root='../data', split='test', download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

Files already downloaded and verified


In [65]:
evaluate_network(net, testloader)

  0%|          | 0/250 [00:00<?, ?it/s]

Evaluating model...


100%|██████████| 250/250 [00:01<00:00, 127.90it/s]

Accuracy: 46 %


46.85

In [21]:
evaluate_network(net, testloader)

  0%|          | 0/250 [00:00<?, ?it/s]

Evaluating model...


100%|██████████| 250/250 [00:01<00:00, 126.64it/s]

Accuracy: 47 %


47.3

# dataset de ataque

In [66]:
from torchvision.datasets import ImageFolder

In [67]:
transform_test = Compose([
    ToTensor(),
    Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = ImageFolder(root='../data/cifar10_attack_v2/', transform=transform_test)
testloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)

In [68]:
evaluate_network(net, testloader)

  0%|          | 0/157 [00:00<?, ?it/s]

Evaluating model...


100%|██████████| 157/157 [00:01<00:00, 122.08it/s]

Accuracy: 78 %


78.68

# dataset gerado

In [10]:
from torchvision.datasets import ImageFolder

In [37]:
transform_test = Compose([
    Resize((32,32)),
    ToTensor(),
    Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

dataset = ImageFolder(root='../data/copycat_generated_dataset/', transform=transform_test)
testloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

In [12]:
evaluate_network(net, testloader)

  0%|          | 0/3125 [00:00<?, ?it/s]

Evaluating model...


100%|██████████| 3125/3125 [00:23<00:00, 131.62it/s]

Accuracy: 47 %


47.707

In [45]:
pred_dict = {x: 0 for x in range(10)}

print('Generating labels from target...')
with torch.no_grad():
    net.eval()
    
    for images, labels in tqdm.tqdm(testloader):
        images = images.to(device)
        
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.cpu().item()
        
        if predicted == labels.cpu().item():
            pred_dict[predicted] += 1

print(pred_dict)

for k in pred_dict:
    print(f"{k}: {(pred_dict[k]/10000):.2f}%")

  0%|          | 0/100000 [00:00<?, ?it/s]

Generating labels from target...


100%|██████████| 100000/100000 [05:59<00:00, 277.97it/s]

{0: 4363, 1: 2750, 2: 2901, 3: 7357, 4: 6042, 5: 4570, 6: 5296, 7: 3902, 8: 6693, 9: 3833}
0: 0.44%
1: 0.28%
2: 0.29%
3: 0.74%
4: 0.60%
5: 0.46%
6: 0.53%
7: 0.39%
8: 0.67%
9: 0.38%


In [31]:
real_labels = np.array([])
pred_labels = np.array([])

print('Generating labels from target...')
with torch.no_grad():
    net.eval()
    
    for images, labels in tqdm.tqdm(testloader):
        real_labels = np.append(real_labels, labels.numpy())
        images = images.to(device)
        
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.cpu().numpy()

        pred_labels = np.append(pred_labels, predicted)

  0%|          | 0/3125 [00:00<?, ?it/s]

Generating labels from target...


100%|██████████| 3125/3125 [00:23<00:00, 133.44it/s]


In [32]:
np.set_printoptions(suppress=True)

In [33]:
(unique, counts) = np.unique(real_labels, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[    0., 10000.],
       [    1., 10000.],
       [    2., 10000.],
       [    3., 10000.],
       [    4., 10000.],
       [    5., 10000.],
       [    6., 10000.],
       [    7., 10000.],
       [    8., 10000.],
       [    9., 10000.]])

In [34]:
(unique, counts) = np.unique(pred_labels, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

array([[    0.,  7823.],
       [    1.,  3461.],
       [    2.,  5433.],
       [    3., 29088.],
       [    4., 12928.],
       [    5., 13667.],
       [    6.,  6187.],
       [    7.,  4263.],
       [    8., 12599.],
       [    9.,  4551.]])